In [17]:
library(ggstatsplot)
library(ggside)
library(dplyr)
library(summarytools)
library(ggfortify)
library(plyr)
library(easyPubMed)

Copied to overview_of_meta_data.html

In [ ]:
Sys.setenv(psi_fig_tables_RData_dir="/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/fig_tables/RData")

In [19]:
ANALYZE_ORIGINAL_DATA = FALSE
if(ANALYZE_ORIGINAL_DATA){
    setwd("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/")
    load("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/data/psi_meta.Rdata")
}else{
    setwd("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi_IRcorrected/")
    load("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi_IRcorrected/data/psi_meta.Rdata")
}



In [ ]:
psi_var_fil <- t(psi_reordered)

In [ ]:
factor_cmp <- function(col_of_interest, feature, p_threshold=0.001, dat=NA, plot=FALSE){
    p <- NA
    rho <- NA
    return(list(p=1.0, rho=0))
    if(var(dat[[col_of_interest]], na.rm =T) ==0){
        return(list(p=1.0, rho=0))
    }
    feature_number <- dat[[feature]][!is.na(dat[[col_of_interest]])] %>% unique() %>% length()
    if((is.character(dat[[feature]]) | is.factor(dat[[feature]])) & feature_number > 1){
        if (feature_number >2){
            p <- kruskal.test(dat[[col_of_interest]] ~ dat[[feature]])$p.value
        }else{
            p <- wilcox.test(dat[[col_of_interest]] ~ dat[[feature]])$p.value
        }
        if(plot & p < p_threshold){
            g <- ggbetweenstats(
              x     = !!rlang::sym(feature),
              y     = !!rlang::sym(col_of_interest),
              data = dat
            )
            print(g)
        }
    }
    if(is.numeric(dat[[feature]])){
        cor_obj <- cor.test(dat[[col_of_interest]], dat[[feature]], method = "spearman", exact = FALSE)
        p <- cor_obj$p.value
        rho <- cor_obj$estimate
        if(plot & p < p_threshold){
            g <- ggscatterstats(
              x     = !!rlang::sym(feature),
              y     = !!rlang::sym(col_of_interest),
              data = dat,
              type = "np" # "np" (nonparametric/spearman)
            )
            print(g)
        }
    }
    return(list(p=p, rho=rho))
    #return(p)
}

In [22]:
# selected_samples is boolean list
# grouping_col is string like 'age_group'
get_assoc_result_for_selected_samples <- function(selected_samples, grouping_col){
    selected_meta <- meta[selected_samples,]
    selected_psi <- psi_var_fil[selected_samples,]
    dim(selected_psi)
    dim(selected_meta)
    dat <- cbind(selected_meta, selected_psi)
    options(warn=-1)
    tissue_gtex_pvals <- sapply(
        colnames(selected_psi), 
        function(x)factor_cmp(x, grouping_col, p_threshold=0.05/dim(selected_psi)[2], dat=dat)
        )
    options(warn=0)
    assoc_pvals <- as.data.frame(t(tissue_gtex_pvals)[,1:2] )
    assoc_result <- data.frame(pval=unlist(assoc_pvals$p), name=rownames(assoc_pvals), estimate=unlist(assoc_pvals$rho))
    assoc_result$gene <- sapply(assoc_result$name, function(x)strsplit(strsplit(x, ":")[[1]][2], '%')[[1]][1])
    assoc_result$exon_num <- sapply(assoc_result$name, function(x)strsplit(x, ":")[[1]][1])
    assoc_result$fdr <- p.adjust(assoc_result$pval, method="BH") 
    assoc_result$signif <- assoc_result$fdr <0.05
    assoc_result
}

In [ ]:
add_delta_mean <- function(assoc_result, dat, feature="age_group"){

    feature_means <- aggregate(dat[,assoc_result$name], by=list(tissue=dat[,feature]), FUN=function(x){mean(x, na.rm=T)})
    feature_sds <- aggregate(dat[,assoc_result$name], by=list(tissue=dat[,feature]), FUN=function(x){sd(x, na.rm=T)})

    assoc_result.1 <- cbind(assoc_result, t(feature_means)[-1,], t(feature_sds)[-1,])
    assoc_result.1.len <- dim(assoc_result.1)[2]
    mean_var_cols <- 8:assoc_result.1.len
    for(i in mean_var_cols){
        assoc_result.1[,i] <- as.numeric(assoc_result.1[,i])
    }
    colnames(assoc_result.1)[mean_var_cols] <- c(paste("mean", feature_means[,1], sep="."), paste("sd", feature_means[,1], sep="."))

    assoc_result.1$delta_mean <- max(abs(assoc_result.1[,8] - assoc_result.1[,9]), 
                                     abs(assoc_result.1[,8] - assoc_result.1[,10]),
                                     abs(assoc_result.1[,9] - assoc_result.1[,10])
                                    )

    delta1 <- abs(assoc_result.1[,8] - assoc_result.1[,9])
    delta2 <- abs(assoc_result.1[,8] - assoc_result.1[,10])
    delta3 <- abs(assoc_result.1[,9] - assoc_result.1[,10])

    assoc_result.1$delta_mean <- apply(cbind(delta1, delta2, delta3), 1, function(x){max(x)})
    assoc_result.sig <- assoc_result.1 %>% filter(signif == TRUE)
    #plot(assoc_result.sig$delta_mean, -log10(assoc_result.sig$fdr))
    #plot(density(assoc_result.sig %>% filter(!is.na(delta_mean)) %>% pull(delta_mean)))
    assoc_result.1
}

In [24]:
feature <- "age_group"
selected_samples <- meta$tissue == "skeletal_muscle" & meta$disease == "normal"


In [25]:
selected_meta <- meta[selected_samples,]
selected_psi <- psi_var_fil[selected_samples,]
dat <- cbind(selected_meta, selected_psi)

In [26]:

skeletal_muscle_age_group_assoc <- get_assoc_result_for_selected_samples(selected_samples, feature)

In [27]:
skeletal_muscle_age_group_assoc.1 <- add_delta_mean(skeletal_muscle_age_group_assoc, dat)

In [28]:
skeletal_muscle_age_group_assoc.1 <- skeletal_muscle_age_group_assoc.1 %>% mutate(gene0=sub("\\|\\w+", "", gene))

In [29]:
skeletal_muscle_age_group_assoc.1

,pval,name,estimate,gene,exon_num,fdr,signif,mean.fetal_neonatal,mean.pediatric,mean.adult,sd.fetal_neonatal,sd.pediatric,sd.adult,delta_mean,gene0
,<dbl>,<chr>,<lgl>,<chr>,<chr>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
001:SPG21%ENST00000204566.7,4.986562e-11,001:SPG21%ENST00000204566.7,NA,SPG21,001,7.278370e-10,TRUE,0.9911541,0.9604835,0.7865225,3.836553e-03,3.337050e-02,1.289697e-01,0.2046316,SPG21
001:SLC25A1%ENST00000215882.10,5.003351e-02,001:SLC25A1%ENST00000215882.10,NA,SLC25A1,001,6.267096e-02,FALSE,0.9312430,0.8896641,0.9207064,4.558191e-02,5.071311e-02,5.017446e-02,0.0415789,SLC25A1
001:PFN1%ENST00000225655.6,1.495900e-03,001:PFN1%ENST00000225655.6,NA,PFN1,001,2.643866e-03,TRUE,0.9618677,0.9301243,0.9574824,1.706153e-02,2.560205e-02,3.182525e-02,0.0317434,PFN1
001:RP11-400F19.8+COASY+NAGLU%ENST00000225927.7,NA,001:RP11-400F19.8+COASY+NAGLU%ENST00000225927.7,NA,RP11-400F19.8+COASY+NAGLU,001,NA,NA,NA,NA,0.8694899,NA,NA,8.498094e-02,NA,RP11-400F19.8+COASY+NAGLU
001:THG1L%ENST00000231198.12+ENST00000521655.1,3.297223e-06,001:THG1L%ENST00000231198.12+ENST00000521655.1,NA,THG1L,001,1.093702e-05,TRUE,0.9861981,0.9388078,0.9737992,8.606012e-03,3.800656e-02,1.541332e-02,0.0473903,THG1L
001:NUP155%ENST00000231498.8,7.554675e-07,001:NUP155%ENST00000231498.8,NA,NUP155,001,2.991865e-06,TRUE,0.9952503,0.9918602,0.9951863,8.498946e-04,1.498252e-03,1.857956e-03,0.0033901,NUP155
001:GNB4%ENST00000232564.8,6.419714e-04,001:GNB4%ENST00000232564.8,NA,GNB4,001,1.237102e-03,TRUE,0.7317487,0.6681920,0.7656166,4.123919e-02,9.034548e-02,9.096409e-02,0.0974246,GNB4
001:NEK9%ENST00000238616.10,2.609240e-01,001:NEK9%ENST00000238616.10,NA,NEK9,001,2.851750e-01,FALSE,0.9583025,0.9747134,0.9665995,2.951409e-02,8.907302e-03,2.030945e-02,0.0164109,NEK9
001:AARS2%ENST00000244571.5,3.424309e-06,001:AARS2%ENST00000244571.5,NA,AARS2,001,1.132589e-05,TRUE,0.9947129,0.9908870,0.9952474,2.190988e-03,2.429231e-03,2.938245e-03,0.0043604,AARS2


In [30]:
write.table(skeletal_muscle_age_group_assoc.1, "skeletal_muscle_age_group_assoc.1.tsv", quote=F, sep="\t", col.names=NA)

## Below is for cardiac vs age groups

In [31]:
feature <- "age_group"
selected_samples <- meta$tissue == "heart" & meta$disease == "normal"
selected_meta <- meta[selected_samples,]
selected_psi <- psi_var_fil[selected_samples,]
dat <- cbind(selected_meta, selected_psi)

In [32]:

cardiac_muscle_age_group_assoc <- get_assoc_result_for_selected_samples(selected_samples, feature)


In [33]:
cardiac_muscle_age_group_assoc.1 <- add_delta_mean(cardiac_muscle_age_group_assoc, dat)

In [34]:
cardiac_muscle_age_group_assoc.1 <- cardiac_muscle_age_group_assoc.1 %>% mutate(gene0=sub("\\|\\w+", "", gene))

In [35]:
write.table(cardiac_muscle_age_group_assoc.1, "cardiac_muscle_age_group_assoc.1.tsv", quote=F, sep="\t", col.names=NA)

In [36]:
save(skeletal_muscle_age_group_assoc.1, 
     cardiac_muscle_age_group_assoc.1, 
     file="data/age_group.assoc.1.Rdata")